# Traitement des données
Pour plus d'information sur les sources de données, lisez le README.md

In [79]:
import pandas as pd
import numpy as np
import duckdb

## 1. Traitement de la liste des communes, des données de l'INSEE sur la population municipale et le nombre de conseillers municipaux,

In [80]:
# Télécharger et importer le fichier des communes au 1er janvier 2025 (à télécharger ici : https://www.insee.fr/fr/information/8377162)
df0 = pd.read_csv("../data/raw_data/v_commune_2025.csv", sep=",", dtype={"COM": "string"})

# Ne garder que les communes seules (TYPECOM == 'COM')
df0 = df0[df0['TYPECOM'] == 'COM']

df0 = df0[['COM', 'LIBELLE']]
df0 = df0.rename(columns={
    "COM": "code_commune",
    "LIBELLE": "libelle"
})

print(df0.shape)
df0.head()

(34875, 2)


,code_commune,libelle
0,01001,L'Abergement-Clémenciat
1,01002,L'Abergement-de-Varey
2,01004,Ambérieu-en-Bugey
3,01005,Ambérieux-en-Dombes
4,01006,Ambléon


In [81]:
#Télécharger et importer le fichier des codes postaux (à télécharger ici: https://www.data.gouv.fr/datasets/base-officielle-des-codes-postaux/)
df0_1 = pd.read_csv("../data/raw_data/codes_postaux.csv", sep=";", encoding="latin-1", dtype={"Code_postal": "string"})
df0_1 = df0_1[["#Code_commune_INSEE", "Code_postal"]]
df0_1 = df0_1.rename(columns={
    "#Code_commune_INSEE": "code_commune",
    "Code_postal": "code_postal"
})

# On retire les doublon et on groupe les code_postaux par code_commune
df0_1 = df0_1.drop_duplicates()
df0_1 = (
    df0_1.groupby("code_commune")["code_postal"]
      .agg(lambda x: sorted(set(x)))
      .reset_index()
)

print(df0_1.shape)

(35007, 2)


In [82]:
# Jointure sur le code commune
df0 = df0.merge(df0_1, on="code_commune", how="left")
df0 = df0.sort_values(by="code_commune")

idx = df0.loc[df0['code_commune'] == '75056', 'code_postal'] = [['75001', '75002', '75003', '75004', '75005', '75006', '75007', '75008', '75009', '75010',
                              '75011', '75012', '75013', '75014', '75015', '75016', '75017', '75018', '75019', '75020']]
idx = df0.loc[df0['code_commune'] == '69123', 'code_postal'] = [['69001', '69002', '69003', '69004', '69005', '69006', '69007', '69008', '69009']]

idx = df0.loc[df0['code_commune'] == '13055', 'code_postal'] = [['13001', '13002', '13003', '13004', '13005', '13006', '13007',
                               '13008', '13009', '13010', '13011', '13012', '13013', '13014', '13015', '13016']]

print(df0.shape)
df0.head()

(34875, 3)


,code_commune,libelle,code_postal
0,01001,L'Abergement-Clémenciat,[01400]
1,01002,L'Abergement-de-Varey,[01640]
2,01004,Ambérieu-en-Bugey,[01500]
3,01005,Ambérieux-en-Dombes,[01330]
4,01006,Ambléon,[01300]


In [83]:
# Importer le fichier population municipale totale (population de référence) (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_POPULATIONS_REFERENCE "Télécharger la totalité du jeu de données")
# Il manque des données sur Mayotte
# j'ai ajouté à la main les données (source wikipedia)
# 12218 Conques-en-Rouergue         --> 1 555 (code commune pas à jour dans ce fichier avant: 12076)
# 14581 Aurseulles                  --> 1 908 (pop sur commune 14011)
# 15031 Celles                      --> 217 (redevient une commune de plein exercice au 1er janvier 2025)
# 15035 Chalinargues                --> 311 (redevient une commune de plein exercice au 1er janvier 2025)
# 15047 Chavagnac                   --> 91 (redevient une commune de plein exercice au 1er janvier 2025)
# 15171 Sainte-Anastasie            --> 124 (redevient une commune de plein exercice au 1er janvier 2025)
# 49126 Orée d'Anjou                --> 16 975 (pop sur la commune 49069)
# 69114 Porte des Pierres Dorées    --> 4 079 (code commune pas à jour dans ce fichier avant: 69159)
# 97601 Acoua 
# 97602 Bandraboua
# 97603 Bandrele
# 97604 Bouéni
# 97605 Chiconi
# 97606 Chirongui
# 97607 Dembeni
# 97608 Dzaoudzi
# 97609 Kani-Kéli
# 097610 Koungou
# 197611 Mamoudzou
# 297612 Mtsamboro
# 397613 M'Tsangamouji
# 497614 Ouangani
# 597615 Pamandzi
# 697616 Sada
# 797617 Tsingoni
df2 = pd.read_csv(r'../data/raw_data/DS_POPULATIONS_REFERENCE_data.csv', sep=';')

# Sélectionner les communes, l'année 2022 et la population municipale (PMUN)
df2 = df2[df2['TIME_PERIOD'] == 2022]
df2 = df2[df2['GEO_OBJECT'] == 'COM']
df2 = df2[df2['POPREF_MEASURE'] == 'PMUN']

# Supprimer les colonnes GEO_OBJECT, TIME_PERIOD, POPREF_MEASURE et FREQ
df2 = df2.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'POPREF_MEASURE', 'FREQ'])

# Renommer la colonne OBS_VALUE en population_municipale
df2 = df2.rename(columns={'OBS_VALUE': 'population_municipale'})

In [84]:
def calculer_conseillers_municipaux(fichier_repartition):
    """
    Calcule le nombre de conseillers municipaux pour chaque commune à partir du fichier de population municipale (df2) et du fichier "conseillers.csv" qui fournit la répartition selon les fourchettes de population.
   
    Parameter:
    -----------
    fichier_repartition : str
       Chemin vers le fichier CSV avec la répartition population/conseillers
       
    Returns:
    --------
    pandas.DataFrame
        DataFrame avec les communes et leur nombre de conseillers
   """
   
    # Charger le fichier
    repartition_df = pd.read_csv(fichier_repartition)
   
    # Charger le dataframe population
    communes_df = df2
   
    # Fonction pour déterminer le nombre de conseillers
    def get_nb_conseillers(population):
        if population == 0:
           return 0
        else:
            for _, row in repartition_df.iterrows():
                if row['population_min'] <= population <= row['population_max']:
                    return row['nombre_de_conseillers']
        return None
   
    # Calculer le nombre de conseillers pour chaque commune
    communes_df['total_conseillers'] = communes_df['population_municipale'].apply(get_nb_conseillers)

    communes_df.loc[communes_df['GEO'] == '75056', 'total_conseillers'] = 163  # Paris
    communes_df.loc[communes_df['GEO'] == '69123', 'total_conseillers'] = 73   # Lyon
    communes_df.loc[communes_df['GEO'] == '13055', 'total_conseillers'] = 101  # Marseille

    return communes_df

In [85]:
# Lancer la fonction avec le fichier de répartition
df2 = calculer_conseillers_municipaux('../data/raw_data/conseillers.csv')
df2 = df2.rename(columns={
    "GEO": "code_commune",
})
df2 = df2.sort_values("code_commune")
print(df2.shape)
df2.head()

(34918, 3)


,code_commune,population_municipale,total_conseillers
13281,01001,859,15
10317,01002,273,11
20215,01004,15554,33
12779,01005,1917,19
15623,01006,114,11


In [86]:
# Merge de df0 et df2
df0 = df0.merge(df2, on="code_commune", how="left")
print(df0.shape)
df0.head()

(34875, 5)


,code_commune,libelle,code_postal,population_municipale,total_conseillers
0,01001,L'Abergement-Clémenciat,[01400],859.0,15.0
1,01002,L'Abergement-de-Varey,[01640],273.0,11.0
2,01004,Ambérieu-en-Bugey,[01500],15554.0,33.0
3,01005,Ambérieux-en-Dombes,[01330],1917.0,19.0
4,01006,Ambléon,[01300],114.0,11.0


In [87]:
df0.loc[df0['code_commune'] == '12218', 'population_municipale'] = 1555
df0.loc[df0['code_commune'] == '12218', 'total_conseillers'] = 19
df0.loc[df0['code_commune'] == '14581', 'population_municipale'] = 1908
df0.loc[df0['code_commune'] == '14581', 'total_conseillers'] = 19
df0.loc[df0['code_commune'] == '15031', 'population_municipale'] = 217
df0.loc[df0['code_commune'] == '15031', 'total_conseillers'] = 11
df0.loc[df0['code_commune'] == '15035', 'population_municipale'] = 311
df0.loc[df0['code_commune'] == '15035', 'total_conseillers'] = 11
df0.loc[df0['code_commune'] == '15047', 'population_municipale'] = 91
df0.loc[df0['code_commune'] == '15047', 'total_conseillers'] = 7
df0.loc[df0['code_commune'] == '15171', 'population_municipale'] = 124
df0.loc[df0['code_commune'] == '15171', 'total_conseillers'] = 11
df0.loc[df0['code_commune'] == '49126', 'population_municipale'] = 16975
df0.loc[df0['code_commune'] == '49126', 'total_conseillers'] = 33
df0.loc[df0['code_commune'] == '69114', 'population_municipale'] = 4079
df0.loc[df0['code_commune'] == '69114', 'total_conseillers'] = 27

## 2. Traitement sur la proportion de locataires et le taux de pauvreté

### 2.A. Traitement de RP (logement) pour calculer la proportion de locataires par commune

In [88]:
# Importer le fichier logement principal (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_RP_LOGEMENT_PRINC "Télécharger la totalité du jeu de données")
# Ici on a pas les données des 4 communes comme avant, ni les 17 communes de mayottes et les 6 communes sans habitant (donc normal)
# Pour certaines communes il manque des données sur les locataire en 2022 TODO : il serait possible d'améliorer en prenant les années d'avant
df3 = pd.read_csv("../data/raw_data/DS_RP_LOGEMENT_PRINC_2022_data.csv", sep=";", low_memory=False, dtype={"GEO": "string"})

# Sélectionner les communes, l'année 2022, la mesure RP_MEASURE = 'DWELLINGS_POPSIZE'
df3 = df3[df3['TIME_PERIOD'] == 2022]
df3 = df3[df3['GEO_OBJECT'] == 'COM']
df3 = df3[df3['RP_MEASURE'] == 'DWELLINGS_POPSIZE']
df3 = df3[df3['TSH'].isin(['211', '212_222', '221', '300', "_T"])]
df3 = df3[df3['OCS'].isin(['DW_MAIN', 'DW_SEC_DW_OCC'])]
df3 = df3[df3['TDW'] == '_T']
df3 = df3[df3['L_STAY'] == '_T']

# Supprimer les colonnes GEO_OBJECT, TIME_PERIOD, OCS, L_STAY, TDW, NRG_SRC, CARS, RP_MEASURE, CARPARK, NOR and BUILD_END
df3 = df3.drop(columns=['GEO_OBJECT', 'OCS', 'L_STAY', 'TDW', 'NRG_SRC','TIME_PERIOD', 'CARS', 'RP_MEASURE', 'CARPARK', 'NOR', 'BUILD_END'])

# Arrondir OBS_VALUE à 'entier le plus proche
df3["OBS_VALUE"] = np.round(df3["OBS_VALUE"])

In [89]:
# # Catégories de locataires
categories_locataires = ['211', '212_222', '221', '300']

# On somme les lignes qui sont categories_locataires
total_locataires = (
    df3[df3['TSH'].isin(categories_locataires)]
    .groupby('GEO')['OBS_VALUE']
    .sum()
    .reset_index()
    .rename(columns={"OBS_VALUE": 'total_locataires'})
)

# on ne garde que la lige avec le total et on fusionne
df3 = df3[df3['TSH'] == '_T']
df3 = df3.merge(total_locataires, on='GEO', how='left')

# Plus besoinde la colonne TSH
df3 = df3.drop(columns=["TSH"])
df3 = df3.sort_values("GEO")
df3 = df3.rename(columns={
    "GEO": "code_commune",
    "OBS_VALUE": "total_loc_et_prop"
})

print(df3.shape)
df3.head()

(34848, 3)


,code_commune,total_loc_et_prop,total_locataires
21798,01001,859.0,112.0
9190,01002,273.0,27.0
6478,01004,15053.0,7939.0
6115,01005,1917.0,447.0
10312,01006,114.0,31.0


In [90]:
# con = duckdb.connect()

# dfb = con.execute("""
#                 SELECT com.code_commune, com.libelle, com.population_municipale
#                 FROM 'data/processed/communes.parquet' as com
#                 LEFT JOIN 'data/processed/df3.parquet' as d
#                 ON com.code_commune = d.code_commune
#                 WHERE d.code_commune IS NULL   
# """).fetch_df()

# pd.set_option("display.max_rows", None)
# print(dfb)

In [91]:
# Merge de df0 et de df3
df0 = df0.merge(df3, on="code_commune", how="left")
print(df0.shape)

(34875, 7)


### 2.B Traitement du taux de pauvreté par commune

In [92]:
# Télécharger et importer le fichier taux de pauvreté (à télécharger ici : https://catalogue-donnees.insee.fr/fr/explorateur/DS_FILOSOFI_AGE_TP_NIVVIE "Télécharger la totalité du jeu de données")
df4 = pd.read_csv("../data/raw_data/DS_FILOSOFI_AGE_TP_NIVVIE_2021_data.csv", sep=";", dtype={"GEO": "string"})


# Filtrer les données
df4 = df4[df4['GEO_OBJECT'] == 'COM']
df4 = df4[df4['AGE_RF'] == '_T']
df4 = df4[df4['FILOSOFI_MEASURE'] == 'PR_MD60']
df4 = df4[df4['CONF_STATUS'] == 'F']
df4 = df4[df4['OBS_STATUS'] == 'A']


# Supprimer les colonnes
df4 = df4.drop(columns=['GEO_OBJECT', 'AGE_RF', 'FILOSOFI_MEASURE', 'UNIT_MEASURE', 'UNIT_MULT', 'CONF_STATUS', 'OBS_STATUS', 'TIME_PERIOD'])

# Renommer les colonnes
df4 = df4.rename(columns={
    'GEO': 'code_commune',
    'OBS_VALUE': 'taux_pauvrete'
})

In [93]:
# Merge de df0 et de df4
df0 = df0.merge(df4, on="code_commune", how="left")
print(df0.shape)

(34875, 8)


### 2.C Export en parquet

In [94]:
# Exporter df0 en parquet
df0 = df0.sort_values('code_commune')
df0.to_parquet('../data/processed/communes.parquet', index=False)

In [95]:
df0

,code_commune,libelle,code_postal,population_municipale,total_conseillers,total_loc_et_prop,total_locataires,taux_pauvrete
0,01001,L'Abergement-Clémenciat,[01400],859.0,15.0,859.0,112.0,NaN
1,01002,L'Abergement-de-Varey,[01640],273.0,11.0,273.0,27.0,NaN
2,01004,Ambérieu-en-Bugey,[01500],15554.0,33.0,15053.0,7939.0,17.0
3,01005,Ambérieux-en-Dombes,[01330],1917.0,19.0,1917.0,447.0,NaN
4,01006,Ambléon,[01300],114.0,11.0,114.0,31.0,NaN
...,...,...,...,...,...,...,...,...
34870,97613,M'Tsangamouji,[97650],NaN,NaN,NaN,NaN,NaN
34871,97614,Ouangani,[97670],NaN,NaN,NaN,NaN,NaN
34872,97615,Pamandzi,[97615],NaN,NaN,NaN,NaN,NaN
34873,97616,Sada,[97640],NaN,NaN,NaN,NaN,NaN


## 3. Traitement de RP (exploitation complémentaire) pour calculer les effectifs de CSP par commune

In [ ]:
# Télécharger et importer le fichier population et csp (à télécharger ici : https://catalogue-donnees.insee.fr/fr/catalogue/recherche/DS_RP_POPULATION_COMP "Télécharger la totalité du jeu de données")
df = pd.read_csv(r'../data/raw_data/DS_RP_POPULATION_COMP_2022_data.csv', sep=';', low_memory=False, dtype={"GEO": "string"})

In [16]:
# Sélection des communes uniquement, de l'année 2022, du sexe _T et de la tranche d'âge 'Y_GE15'
df1 = df[df['GEO_OBJECT'] == 'COM']
df1 = df1[df1['TIME_PERIOD'] == 2022]
df1 = df1[df1['SEX'] == '_T']
df1 = df1[df1['AGE'] == 'Y_GE15']

In [17]:
# Drop des colonnes inutiles
df1 = df1.drop(columns=['GEO_OBJECT', 'TIME_PERIOD', 'SEX', 'RP_MEASURE', 'AGE'])

In [18]:
# Arrondis les valeurs de OBS à l'entier le plus proche
df1["OBS_VALUE"] = np.round(df1["OBS_VALUE"])

In [19]:
df1 = df1.rename(columns={
    "GEO": "code_commune",
    "PCS": "code_csp",
    "OBS_VALUE": "population_csp"
})
df1 = df1.sort_values("code_commune")
print(df1.shape)
df1.head()

(286188, 3)


,code_commune,code_csp,population_csp
1485260,01001,9,44.0
1482599,01001,5,108.0
1483016,01001,4,103.0
2539648,01001,_T,692.0
1506118,01001,3,74.0


In [ ]:
# Exporter df1 en parquet
# il est important de noter qu'il manque des données sur 31 communes mais cela concerne les
# communes de mayottes on a toujours pas de données, certaines communes avec moins de 10 habitant (mais pas toutes),
# et les 4 communes qui redeviennet des communes à part entière en 2025
df1.to_parquet('../data/processed/population_communes_csp_2022.parquet', index=False)

In [ ]:
# Permet de créer un .parquet pour avoir les libelles des CSP
con = duckdb.connect()

data = {
    '1': "Agriculteurs",
    '2': "Artisans - commerçants et chefs d'entreprise",
    '3': "Cadres et professions intellectuelles supérieures",
    '4': "Professions intermédiaires",
    '5': "Employés",
    '6': "Ouvriers",
    '7': "Retraités",
    '9': "Autres inactifs"
}

dfc = pd.DataFrame([
    {"code_csp": k, "libelle_csp": v}
    for k, v in data.items()
])

dfc.to_parquet("../data/processed/csp.parquet", index=False)

In [12]:
df = pd.read_parquet("../data/processed/communes.parquet")

In [ ]:
df